In [1]:
import pandas as pd
import requests
import json
from config import api_key
from config import shared_secret
import time

In [2]:
top_500_df = pd.read_csv('albumlist.csv')

In [3]:
top_500_df.loc[:,'Artist'].nunique()

289

In [4]:
top_500_df.loc[:,['Artist','Album']].groupby('Artist').count().sort_values('Album',ascending=False)

,Album
Artist,
The Beatles,10
Bob Dylan,10
The Rolling Stones,10
Bruce Springsteen,8
The Who,7
Radiohead,5
U2,5
Led Zeppelin,5
Elton John,5


In [5]:
genre_list = ['Rock','Pop','Funk / Soul', 'Blues', 'Jazz', 'Folk', 'World', 'Country', 'Classical', 'Stage & Screen', 'Reggae','Hip Hop','Electronic','Latin']
for genre in genre_list:
    top_500_df[genre] = 0

In [6]:
for index,row in top_500_df.iterrows():
    temp_string = str(row['Genre'])
    for genre in genre_list:
        if genre in temp_string:
            top_500_df.at[index,genre]=1

In [7]:
top_500_df[genre_list].sum().sort_values(ascending=False)

Rock              351
Funk / Soul        80
Pop                54
Electronic         45
Hip Hop            38
Blues              38
Country            35
World              35
Folk               35
Jazz               19
Reggae             12
Stage & Screen      8
Latin               5
Classical           1
dtype: int64

In [8]:
top_500_df['num_genres'] = 0
for index,row in top_500_df.iterrows():
    temp_genre_list = str(row['Genre']).split(',')
    top_500_df.at[index,'num_genres'] = len(temp_genre_list)

In [9]:
top_500_df.groupby('num_genres').count().loc[:,'Genre']

num_genres
1    345
2     99
3     28
4     18
5      5
6      3
7      2
Name: Genre, dtype: int64

In [10]:
[top_500_df['Year'].min(),top_500_df['Year'].max()]

[1955, 2011]

In [11]:
decades_list = ['1950s','1960s','1970s','1980s','1990s','2000s','2010s']
for decade in decades_list:
    top_500_df[decade]=0

In [12]:
for index,row in top_500_df.iterrows():
    year = row['Year']
    if year<1960:
        top_500_df.at[index,'1950s']=1
    elif year>1959 and year <1970:
        top_500_df.at[index,'1960s']=1
    elif year>1969 and year <1980:
        top_500_df.at[index,'1970s']=1
    elif year>1979 and year <1990:
        top_500_df.at[index,'1980s']=1
    elif year>1989 and year <2000:
        top_500_df.at[index,'1990s']=1
    elif year>1999 and year <2010:
        top_500_df.at[index,'2000s']=1
    elif year>2010:
        top_500_df.at[index,'2010s']=1

In [13]:
top_500_df[decades_list].sum().sort_values(ascending=False)

1970s    186
1960s    105
1980s     85
1990s     72
2000s     40
1950s     10
2010s      1
dtype: int64

In [14]:
artists = top_500_df.loc[:,'Artist'].unique()
artists_lower = []
for artist in artists:
    artist_lower = artist.lower()
    if '/' in artist and artist_lower!='ac/dc':
        artist_modified_list = artist_lower.split('/')
        for artist_modified in artist_modified_list:
            artists_lower.append(artist_modified)
    else:
        artists_lower.append(artist_lower)
        
rolling_stone_artists =[]

########################################################
#
#need metadata for band X
for artist in artists_lower:
    if artist != 'x':
        rolling_stone_artists.append(artist)
        
rolling_stone_artists

['the beatles',
 'the beach boys',
 'bob dylan',
 'marvin gaye',
 'the rolling stones',
 'the clash',
 'elvis presley',
 'miles davis',
 'the velvet underground',
 'the jimi hendrix experience',
 'nirvana',
 'bruce springsteen',
 'van morrison',
 'michael jackson',
 'chuck berry',
 'robert johnson',
 'john lennon',
 'plastic ono band',
 'stevie wonder',
 'james brown',
 'fleetwood mac',
 'u2',
 'the who',
 'led zeppelin',
 'joni mitchell',
 'ramones',
 'the band',
 'david bowie',
 'carole king',
 'eagles',
 'muddy waters',
 'love',
 'sex pistols',
 'the doors',
 'pink floyd',
 'patti smith',
 'bob marley & the wailers',
 'john coltrane',
 'public enemy',
 'the allman brothers band',
 'little richard',
 'simon & garfunkel',
 'al green',
 'ray charles',
 'creedence clearwater revival',
 'captain beefheart & his magic band',
 'sly & the family stone',
 "guns n' roses",
 'phil spector',
 'radiohead',
 'billy joel',
 'paul simon',
 'curtis mayfield',
 'neil young',
 'prince and the revoluti

In [15]:
billboard_csv = 'billboard_top_100_songs.csv'
billboard_df = pd.read_csv(billboard_csv)
billboard_df.head()

,pos,last week,peak,weeks on chart,title,artist,chart entry date,entry position,overall peak,overall weeks on chart,chart date
0,1,1,1.0,15.0,Uptown Funk!,MARK RONSON featuring BRUNO MARS,41972.0,65.0,1.0,15.0,20150307
1,2,2,2.0,20.0,Thinking Out Loud,ED SHEERAN,41937.0,69.0,2.0,20.0,20150307
2,3,6,3.0,7.0,Love Me Like You Do,ELLIE GOULDING,42028.0,45.0,3.0,7.0,20150307
3,4,5,4.0,6.0,Sugar,MAROON 5,42035.0,8.0,4.0,6.0,20150307
4,5,3,2.0,28.0,Take Me To Church,HOZIER,41881.0,96.0,2.0,28.0,20150307


In [16]:
billboard_columns = billboard_df.columns.values
billboard_df_filtered = pd.DataFrame(columns = billboard_columns)
index_list = []
artist_index_list = []
for index,row in billboard_df.iterrows():
    artist = row['artist']
    for rolling_stone_artist in rolling_stone_artists:
        if artist.lower().find(rolling_stone_artist) !=-1:
            index_list.append(index)
            artist_index_list.append(rolling_stone_artist)

In [17]:
billboard_df_filtered = billboard_df.iloc[index_list,:]
billboard_df_filtered = billboard_df_filtered.reset_index()
len(billboard_df_filtered)

49284

In [18]:
billboard_df_filtered['rolling stone artist'] = pd.Series(artist_index_list)
billboard_df_filtered

,index,pos,last week,peak,weeks on chart,title,artist,chart entry date,entry position,overall peak,overall weeks on chart,chart date,rolling stone artist
0,5,6,4,4.0,5.0,FourFiveSeconds,RIHANNA & KANYE WEST & PAUL McCARTNEY,42042.0,54.0,4.0,5.0,20150307,kanye west
1,15,16,19,12.0,17.0,Only,"NICKI MINAJ featuring DRAKE, LIL WAYNE & CHRIS...",41958.0,54.0,12.0,17.0,20150307,lil wayne
2,16,17,22,17.0,5.0,Truffle Butter,NICKI MINAJ featuring DRAKE & LIL WAYNE,42042.0,71.0,17.0,5.0,20150307,lil wayne
3,17,18,18,7.0,24.0,Jealous,NICK JONAS,41909.0,78.0,7.0,24.0,20150307,nas
4,35,36,26,20.0,14.0,CoCo,O.T. GENASIS,41979.0,77.0,20.0,14.0,20150307,nas
5,41,42,56,42.0,4.0,Chains,NICK JONAS,42049.0,92.0,42.0,4.0,20150307,nas
6,48,49,40,12.0,22.0,Tuesday,I LOVE MAKONNEN featuring DRAKE,41923.0,90.0,12.0,22.0,20150307,love
7,73,74,45,45.0,4.0,Only One,KANYE WEST featuring PAUL McCARTNEY,42049.0,68.0,45.0,4.0,20150307,kanye west
8,74,75,52,43.0,9.0,Stuck On A Feeling,PRINCE ROYCE featuring SNOOP DOGG,42014.0,93.0,43.0,9.0,20150307,prince
9,76,77,73,12.0,13.0,The Hanging Tree,JAMES NEWTON HOWARD featuring JENNIFER LAWRENCE,41986.0,12.0,12.0,13.0,20150307,war


In [19]:
billboard_top_songs_counts = billboard_df_filtered["title"].value_counts()
billboard_top_songs_counts

Forever                                                         94
Runaway                                                         88
Angel                                                           69
Smooth                                                          58
Holiday                                                         56
Heartbreak Hotel                                                55
Don't Let The Sun Go Down On Me                                 55
Having A Party                                                  55
If                                                              54
Together Again                                                  54
If I Die Young                                                  53
Creep                                                           52
Viva La Vida                                                    51
Scream / Childhood                                              51
Mockingbird                                                   

In [20]:
billboard_top_songs = set(billboard_df_filtered["title"])

In [24]:
unique_song_index = []
for song in billboard_top_songs:
    filter_by_title_df = billboard_df_filtered[billboard_df_filtered['title']==song].sort_values('index')
    artist_temp_list = [filter_by_title_df.iloc[0]['artist']]
    unique_song_index.append(index)
    for index,row in filter_by_title_df.iterrows():
        if row['artist'] not in artist_temp_list:
            artist_temp_list.append(row['artist'])
            unique_song_index.append(index)

In [25]:
billboard_top_songs_df = billboard_df_filtered.iloc[unique_song_index,:]
billboard_top_songs_df = billboard_top_songs_df.sort_values('weeks on chart',ascending=False).reset_index()
billboard_top_songs_df

,level_0,index,pos,last week,peak,weeks on chart,title,artist,chart entry date,entry position,overall peak,overall weeks on chart,chart date,rolling stone artist
0,12393,114548,49,49,7.0,31.0,Real Love,MARY J. BLIGE,33845.0,84.0,7.0,31.0,19930327,mary j. blige
1,47850,321016,97,88,1.0,30.0,All Shook Up,ELVIS PRESLEY,20906.0,26.0,1.0,30.0,19571012,elvis presley
2,48468,327013,94,96,1.0,27.0,Heartbreak Hotel,ELVIS PRESLEY,20507.0,68.0,1.0,27.0,19560822,elvis presley
3,27633,195116,97,92,3.0,27.0,Just The Way You Are,BILLY JOEL,28441.0,89.0,3.0,27.0,19780513,billy joel
4,48232,324702,83,58,1.0,27.0,Don't Be Cruel,ELVIS PRESLEY,20668.0,28.0,1.0,27.0,19570130,elvis presley
5,47429,318502,83,74,1.0,26.0,You Send Me,SAM COOKE,21105.0,96.0,1.0,26.0,19580405,sam cooke
6,8065,88437,38,32,1.0,25.0,Candle In The Wind 1997 / Something About The ...,ELTON JOHN,35714.0,1.0,1.0,42.0,19980328,elton john
7,21205,161693,94,87,16.0,24.0,Holiday,MADONNA,30597.0,108.0,16.0,24.0,19840317,madonna
8,6326,73743,44,29,1.0,24.0,Music,MADONNA,36750.0,41.0,1.0,24.0,20010120,madonna
9,9546,99348,49,43,3.0,24.0,Runaway,JANET JACKSON,34958.0,6.0,3.0,24.0,19960224,janet


In [26]:
billboard_top_songs_df['rolling stone artist'].value_counts()

war                              335
elvis presley                    151
frank sinatra                    130
james brown                      123
nas                              121
lil wayne                        107
ray charles                      107
love                              92
the beatles                       78
elton john                        71
aretha franklin                   70
prince                            69
jackie wilson                     68
the temptations                   67
stevie wonder                     65
marvin gaye                       63
the beach boys                    61
the rolling stones                60
neil diamond                      57
johnny cash                       57
madonna                           57
kanye west                        56
sam cooke                         55
eminem                            51
michael jackson                   50
b.b. king                         44
billy joel                        43
t

In [27]:
billboard_top_songs_df[billboard_top_songs_df['artist']=="THE BAND PERRY"]

,level_0,index,pos,last week,peak,weeks on chart,title,artist,chart entry date,entry position,overall peak,overall weeks on chart,chart date,rolling stone artist
461,2182,24191,92,NEW,92.0,1.0,If I Die Young,THE BAND PERRY,40383.0,92.0,19.0,28.0,20100724,the band
825,518,7096,97,NEW,97.0,1.0,Don't Let Me Be Lonely,THE BAND PERRY,41580.0,97.0,59.0,16.0,20131102,the band
988,1127,14699,100,NEW,100.0,1.0,Postcard From Paris,THE BAND PERRY,41048.0,100.0,100.0,1.0,20120519,the band
1900,1863,20779,80,NEW,80.0,1.0,You Lie,THE BAND PERRY,40621.0,80.0,42.0,20.0,20110319,the band
1903,897,12052,53,NEW,53.0,1.0,Better Dig Two,THE BAND PERRY,41230.0,53.0,28.0,20.0,20121117,the band
2779,1523,18094,95,NEW,95.0,1.0,All Your Life,THE BAND PERRY,40810.0,95.0,37.0,23.0,20110924,the band
4092,295,4093,94,NEW,94.0,1.0,Chainsaw,THE BAND PERRY,41790.0,94.0,86.0,9.0,20140531,the band
4483,759,10186,87,NEW,87.0,1.0,DONE.,THE BAND PERRY,41363.0,87.0,43.0,20.0,20130330,the band


In [28]:
response_list = []
for index,row in billboard_top_songs_df.iterrows():
    artist_name = row['artist']
    song_name = row['title']
    last_fm_url = f'http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={api_key}&artist={artist_name}&track={song_name}&format=json'
    response = requests.get(last_fm_url)
    response_list.append(response)
    print(f'Processing song # {index}: {song_name}, by: {artist_name}: ')
    time.sleep(0.200)        

Processing song # 0: Real Love, by: MARY J. BLIGE: 
Processing song # 1: All Shook Up, by: ELVIS PRESLEY: 
Processing song # 2: Heartbreak Hotel, by: ELVIS PRESLEY: 
Processing song # 3: Just The Way You Are, by: BILLY JOEL: 
Processing song # 4: Don't Be Cruel, by: ELVIS PRESLEY: 
Processing song # 5: You Send Me, by: SAM COOKE: 
Processing song # 6: Candle In The Wind 1997 / Something About The Way You Look T, by: ELTON JOHN: 
Processing song # 7: Holiday, by: MADONNA: 
Processing song # 8: Music, by: MADONNA: 
Processing song # 9: Runaway, by: JANET JACKSON: 
Processing song # 10: Up On The Roof, by: THE DRIFTERS: 
Processing song # 11: Don't Know Much, by: LINDA RONSTADT & AARON NEVILLE: 
Processing song # 12: Take It To The Limit, by: THE EAGLES: 
Processing song # 13: Peggy Sue, by: BUDDY HOLLY: 
Processing song # 14: The One, by: ELTON JOHN: 
Processing song # 15: Ring My Bell, by: ANITA WARD: 
Processing song # 16: Amazing, by: AEROSMITH: 
Processing song # 17: Dream On, by: AE

Processing song # 137: Summertime, by: BILLY STEWART: 
Processing song # 138: (I Know) I'm Losing You, by: THE TEMPTATIONS: 
Processing song # 139: Eleanor Rigby, by: RAY CHARLES: 
Processing song # 140: It's All Over Now, by: THE ROLLING STONES: 
Processing song # 141: Baby I'm Burnin', by: DOLLY PARTON: 
Processing song # 142: You, by: GEORGE HARRISON: 
Processing song # 143: It's The End Of The World As We Know It (And I Feel Fine), by: R.E.M.: 
Processing song # 144: Border Song, by: ELTON JOHN: 
Processing song # 145: Girl, by: THE TEMPTATIONS: 
Processing song # 146: Give It Up, by: THE STEVE MILLER BAND: 
Processing song # 147: My Way, by: FRANK SINATRA: 
Processing song # 148: Jump, by: LOVERBOY: 
Processing song # 149: What's It Gonna Be, by: DUSTY SPRINGFIELD: 
Processing song # 150: Power, by: THE TEMPTATIONS: 
Processing song # 151: Baby I Love You, by: THE RONETTES: 
Processing song # 152: Don't Ask Me Why, by: ELVIS PRESLEY: 
Processing song # 153: Friends, by: ELTON JOHN

Processing song # 267: Little Drummer Boy, by: JOHNNY CASH: 
Processing song # 268: Guess Who, by: DUSTY SPRINGFIELD: 
Processing song # 269: Pops We Love You, by: DIANA ROSS & STEVIE WONDER & MARVIN GAYE: 
Processing song # 270: One Chain, by: SANTANA: 
Processing song # 271: There I've Said It Again, by: SAM COOKE: 
Processing song # 272: Crying In The Chapel, by: LITTLE RICHARD: 
Processing song # 273: Let's Move A Groove, by: JOHNNY NASH: 
Processing song # 274: River Deep - Mountain High, by: IKE & TINA TURNER: 
Processing song # 275: You Got It, by: ETTA JAMES: 
Processing song # 276: Bewildered, by: JAMES BROWN AND THE FAMOUS FLAMES: 
Processing song # 277: 7, by: PRINCE & THE NEW POWER GENERATION: 
Processing song # 278: No Scrubs, by: TLC: 
Processing song # 279: I Got A Thing You Got A Thing Everybody's Got A Thing, by: FUNKADELIC: 
Processing song # 280: New Day, by: PATTI LaBELLE: 
Processing song # 281: I'll Take You Where's The Music Playing, by: THE DRIFTERS: 
Processing

Processing song # 390: The Look Of Love, by: FRANK SINATRA: 
Processing song # 391: Kansas City, by: LITTLE RICHARD: 
Processing song # 392: I Can't Wait, by: SLEEPY BROWN featuring OUTKAST: 
Processing song # 393: The Girl From Ipanema, by: STAN GETZ / ASTRUD GILBERTO: 
Processing song # 394: I've Lost You, by: JACKIE WILSON: 
Processing song # 395: La Montana (If She Should Come To You), by: FRANK DEVOL AND HIS RAINBOW STRINGS: 
Processing song # 396: Crying Time, by: RAY CHARLES with THE RAELETTS: 
Processing song # 397: I'm On Fire, by: JERRY LEE LEWIS: 
Processing song # 398: Touch Me, by: WILLIE NELSON: 
Processing song # 399: Still Dre, by: DR. DRE: 
Processing song # 400: Long Gone Lonesome Blues, by: HANK WILLIAMS JR.: 
Processing song # 401: Why, by: THE BEATLES with TONY SHERIDAN: 
Processing song # 402: Loving You, by: ELVIS PRESLEY: 
Processing song # 403: Freedom Of Choice, by: DEVO: 
Processing song # 404: Peek-A-Boo, by: DEVO: 
Processing song # 405: Part-Time Love, by:

Processing song # 519: Ol MacDonald, by: FRANK SINATRA: 
Processing song # 520: Smooth Criminal, by: MICHAEL JACKSON: 
Processing song # 521: Sweet Little Rock And Roll, by: CHUCK BERRY: 
Processing song # 522: Goin' Down (On The Road To L.A.), by: TERRY BLACK & LAUREL WARD: 
Processing song # 523: Hide Your Heart, by: KISS: 
Processing song # 524: Love Me Two Times, by: THE DOORS: 
Processing song # 525: My Wish Came True, by: ELVIS PRESLEY: 
Processing song # 526: It's All In The Game, by: TOMMY EDWARDS: 
Processing song # 527: Turn Turn Turn, by: THE BYRDS: 
Processing song # 528: The Long Run, by: THE EAGLES: 
Processing song # 529: Islands In The Stream, by: KENNY ROGERS with DOLLY PARTON: 
Processing song # 530: Time Out Of Mind, by: STEELY DAN: 
Processing song # 531: Big Yellow Taxi (Live), by: JONI MITCHELL: 
Processing song # 532: Five Feet High And Risin', by: JOHNNY CASH: 
Processing song # 533: To Make You Feel My Love, by: BILLY JOEL: 
Processing song # 534: Come On, by: 

...

Processing song # 4042: Bo Diddley, by: BUDDY HOLLY: 
Processing song # 4043: You're All I Need To Get By, by: MARVIN GAYE & TAMMI TERRELL: 
Processing song # 4044: The Motto, by: DRAKE featuring LIL WAYNE: 
Processing song # 4045: Band On The Run, by: PAUL McCARTNEY & WINGS: 
Processing song # 4046: Right Above It, by: LIL WAYNE featuring DRAKE: 
Processing song # 4047: Save Me, by: FLEETWOOD MAC: 
Processing song # 4048: With Or Without You, by: U2: 
Processing song # 4049: What Have You Done For Me Lately, by: JANET JACKSON: 
Processing song # 4050: I Don't Want To Spoil The Party, by: THE BEATLES: 
Processing song # 4051: I'll Cry Instead, by: THE BEATLES: 
Processing song # 4052: True Love, by: ELTON JOHN & KIKI DEE: 
Processing song # 4053: Lost, by: GORILLA ZOE featuring LIL WAYNE: 
Processing song # 4054: Maybe Baby, by: THE CRICKETS: 
Processing song # 4055: Blue Money, by: VAN MORRISON: 
Processing song # 4056: Isn't It About Time, by: STEPHEN STILLS & MANASSAS: 
Processing s

Processing song # 4164: The Emperor's New Clothes, by: SINEAD O'CONNOR: 
Processing song # 4165: Saturday Night At The Movies, by: THE DRIFTERS: 
Processing song # 4166: Lady, by: D'ANGELO: 
Processing song # 4167: This Old Heart Of Mine, by: ROD STEWART: 
Processing song # 4168: Blue Shadows, by: B.B. KING: 
Processing song # 4169: Hey Love, by: STEVIE WONDER: 
Processing song # 4170: Feels So Good, by: VAN HALEN: 
Processing song # 4171: And So It Goes, by: BILLY JOEL: 
Processing song # 4172: S.O.S., by: ABBA: 
Processing song # 4173: I'll Be There For You / You're All I Need To Get By, by: METHOD MAN / MARY J. BLIGE: 
Processing song # 4174: What Do I Care, by: JOHNNY CASH: 
Processing song # 4175: White Room, by: CREAM: 
Processing song # 4176: Careless Whisper, by: GEORGE MICHAEL: 
Processing song # 4177: Don't Change On Me, by: RAY CHARLES: 
Processing song # 4178: Streets Of Philadelphia, by: BRUCE SPRINGSTEEN: 
Processing song # 4179: Stand Tall, by: THE O'JAYS: 
Processing so

Processing song # 4291: Infatuation, by: ROD STEWART: 
Processing song # 4292: (Oh) Pretty Woman, by: VAN HALEN: 
Processing song # 4293: Hey Ma, by: CAM'RON feat JULEZ SANTANA, FREEKEY ZEKEY & TOYA: 
Processing song # 4294: Bad Love, by: ERIC CLAPTON: 
Processing song # 4295: Just For A Thrill, by: ARETHA FRANKLIN: 
Processing song # 4296: The Waiting, by: TOM PETTY AND THE HEARTBREAKERS: 
Processing song # 4297: Drive, by: THE CARS: 
Processing song # 4298: Pretending, by: ERIC CLAPTON: 
Processing song # 4299: Yesterday Once More, by: THE CARPENTERS: 
Processing song # 4300: Throw That, by: SLAUGHTERHOUSE featuring EMINEM: 
Processing song # 4301: Next Door To The Blues, by: ETTA JAMES: 
Processing song # 4302: Violet Hill, by: COLDPLAY: 
Processing song # 4303: What's Going On, by: MARVIN GAYE: 
Processing song # 4304: Wherever Would I Be, by: CHEAP TRICK: 
Processing song # 4305: That Girl Could Sing, by: JACKSON BROWNE: 
Processing song # 4306: It Hurts Me, by: ELVIS PRESLEY: 
Pr

Processing song # 4419: Waiting For That Day, by: GEORGE MICHAEL: 
Processing song # 4420: Master Of Eyes (The Deepness Of Your Eyes), by: ARETHA FRANKLIN: 
Processing song # 4421: Rock-A-Hula Baby (Twist Special), by: ELVIS PRESLEY: 
Processing song # 4422: I'm Going In, by: DRAKE featuring LIL WAYNE & YOUNG JEEZY: 
Processing song # 4423: Dolores, by: TOMMY DORSEY & HIS ORCHESTRA / FRANK SINATRA & PIED PIPERS: 
Processing song # 4424: Fun, Fun, Fun, by: THE BEACH BOYS: 
Processing song # 4425: Ask Me, by: ELVIS PRESLEY: 
Processing song # 4426: Hard Headed Woman, by: ELVIS PRESLEY: 
Processing song # 4427: I'm Into You, by: JENNIFER LOPEZ featuring LIL WAYNE: 
Processing song # 4428: Dancing Bear, by: THE MAMAS AND THE PAPAS: 
Processing song # 4429: Brown Sugar, by: THE ROLLING STONES: 
Processing song # 4430: Guts Over Fear, by: EMINEM featuring SIA: 
Processing song # 4431: Money For Nothing, by: DIRE STRAITS: 
Processing song # 4432: 3 Peat, by: LIL WAYNE: 
Processing song # 4433

Processing song # 4541: It's Too Late / I Feel The Earth Move, by: CAROLE KING: 
Processing song # 4542: So What The Fuss, by: STEVIE WONDER: 
Processing song # 4543: Hey Girl, by: THE MAMAS AND THE PAPAS: 
Processing song # 4544: Don't Look Any Further, by: DENNIS EDWARDS: 
Processing song # 4545: I've Cried My Last Tear, by: THE O'JAYS: 
Processing song # 4546: The Right Time, by: RAY CHARLES: 
Processing song # 4547: Leather And Lace, by: STEVIE NICKS with DON HENLEY: 
Processing song # 4548: Break Up 2 Make Ups, by: METHOD MAN featuring D'ANGELO: 
Processing song # 4549: (Sitin' On) The Dock Of The Bay, by: OTIS REDDING: 
Processing song # 4550: Baby Can I Hold You, by: TRACY CHAPMAN: 
Processing song # 4551: Bridging The Gap, by: NAS: 
Processing song # 4552: You're A Friend Of Mine, by: CLARENCE CLEMONS & JACKSON BROWNE: 
Processing song # 4553: Maria, by: BLONDIE: 
Processing song # 4554: Oye Como Va, by: SANTANA: 
Processing song # 4555: Tired Of Waiting For You, by: THE KINKS:

In [33]:
index_except_list = [1467,4481,4579]
top_100_albums_list = []
for index,response in enumerate(response_list):
    artist_name = billboard_top_songs_df.loc[index,'artist']
    song_name = billboard_top_songs_df.loc[index,'title']
    print(f'Processing song # {index}: {song_name}, by: {artist_name}: ')
    if index not in index_except_list:
        response_json = response.json()
        try:
            album = response_json['track']['album']['title']
            artist = response_json['track']['artist']['name']
            song = response_json['track']['name']
        except (KeyError,AttributeError,TimeoutError,json.decoder.JSONDecodeError) as e:
            album = ''
            artist = artist_name
            song = song_name
            print(e)
    else:
        artist = artist_name
        song = song_name
    album_object = {'album':album,'artist':artist,'song':song,'index':index}
    top_100_albums_list.append(album_object)
len(top_100_albums_list)

Processing song # 0: Real Love, by: MARY J. BLIGE: 
Processing song # 1: All Shook Up, by: ELVIS PRESLEY: 
Processing song # 2: Heartbreak Hotel, by: ELVIS PRESLEY: 
Processing song # 3: Just The Way You Are, by: BILLY JOEL: 
Processing song # 4: Don't Be Cruel, by: ELVIS PRESLEY: 
Processing song # 5: You Send Me, by: SAM COOKE: 
Processing song # 6: Candle In The Wind 1997 / Something About The Way You Look T, by: ELTON JOHN: 
'track'
Processing song # 7: Holiday, by: MADONNA: 
Processing song # 8: Music, by: MADONNA: 
Processing song # 9: Runaway, by: JANET JACKSON: 
Processing song # 10: Up On The Roof, by: THE DRIFTERS: 
Processing song # 11: Don't Know Much, by: LINDA RONSTADT & AARON NEVILLE: 
'track'
Processing song # 12: Take It To The Limit, by: THE EAGLES: 
Processing song # 13: Peggy Sue, by: BUDDY HOLLY: 
Processing song # 14: The One, by: ELTON JOHN: 
Processing song # 15: Ring My Bell, by: ANITA WARD: 
Processing song # 16: Amazing, by: AEROSMITH: 
Processing song # 17: 

Processing song # 910: Roman's Revenge, by: NICKI MINAJ featuring EMINEM: 
Processing song # 911: Don't Look Back, by: THE TEMPTATIONS: 
Processing song # 912: I Feel So Bad, by: ELVIS PRESLEY: 
Processing song # 913: Frederick, by: THE PATTI SMITH GROUP: 
Processing song # 914: I'm Comin' On Back To You, by: JACKIE WILSON: 
Processing song # 915: Blue Suede Shoes (From The Elvis Presley EP), by: ELVIS PRESLEY: 
'track'
Processing song # 916: Suzie Q (Part1), by: CREEDENCE CLEARWATER REVIVAL: 
'album'
Processing song # 917: Mamma Mia, by: ABBA: 
Processing song # 918: Hell Breaks Loose, by: EMINEM featuring DR. DRE: 
Processing song # 919: You Are My Sunshine, by: RAY CHARLES: 
Processing song # 920: So Excited, by: B.B. KING: 
Processing song # 921: You're My Everything, by: THE TEMPTATIONS: 
Processing song # 922: Bow Chicka Wow Wow, by: MIKE POSNER featuring LIL WAYNE: 
Processing song # 923: Anthony Boy, by: CHUCK BERRY: 
Processing song # 924: Every Day I Have The Blues, by: BILLY

Processing song # 2611: Stronger, by: KANYE WEST: 
Processing song # 2612: Electrical Storm, by: U2: 
Processing song # 2613: Miss You Much, by: JANET JACKSON: 
Processing song # 2614: Tear The Roof Off The Sucker, by: PARLIAMENT: 
Processing song # 2615: Sunday, Monday Or Always, by: FRANK SINATRA & THE BOBBY TUCKER SINGERS: 
'track'
Processing song # 2616: Oh Boy, by: CAM'RON featuring JUELZ SANTANA: 
'album'
Processing song # 2617: I Want You To Want Me, by: CHEAP TRICK: 
Processing song # 2618: Baby What I Mean, by: THE DRIFTERS: 
Processing song # 2619: Guerrilla Radio, by: RAGE AGAINST THE MACHINE: 
Processing song # 2620: All Cried Out, by: DUSTY SPRINGFIELD: 
Processing song # 2621: She Loves You, by: THE BEATLES: 
Processing song # 2622: Baby Don't You Cry, by: RAY CHARLES: 
Processing song # 2623: If You Don't Come Back, by: THE DRIFTERS: 
Processing song # 2624: Such A Night, by: ELVIS PRESLEY: 
Processing song # 2625: Hello, I Love You, by: THE DOORS: 
Processing song # 262

4599

In [34]:
billboard_top_songs_df['album'] = ''
for index,album_object in enumerate(top_100_albums_list):
    if album_object['album'] != '':
        billboard_top_songs_df.at[index,'album'] = album_object['album']
        
billboard_top_songs_df

,level_0,index,pos,last week,peak,weeks on chart,title,artist,chart entry date,entry position,overall peak,overall weeks on chart,chart date,rolling stone artist,album
0,12393,114548,49,49,7.0,31.0,Real Love,MARY J. BLIGE,33845.0,84.0,7.0,31.0,19930327,mary j. blige,Reflections (A Retrospective)
1,47850,321016,97,88,1.0,30.0,All Shook Up,ELVIS PRESLEY,20906.0,26.0,1.0,30.0,19571012,elvis presley,ELV1S 30 #1 Hits
2,48468,327013,94,96,1.0,27.0,Heartbreak Hotel,ELVIS PRESLEY,20507.0,68.0,1.0,27.0,19560822,elvis presley,ELV1S 30 #1 Hits
3,27633,195116,97,92,3.0,27.0,Just The Way You Are,BILLY JOEL,28441.0,89.0,3.0,27.0,19780513,billy joel,The Stranger
4,48232,324702,83,58,1.0,27.0,Don't Be Cruel,ELVIS PRESLEY,20668.0,28.0,1.0,27.0,19570130,elvis presley,ELV1S 30 #1 Hits
5,47429,318502,83,74,1.0,26.0,You Send Me,SAM COOKE,21105.0,96.0,1.0,26.0,19580405,sam cooke,The Man and His Music
6,8065,88437,38,32,1.0,25.0,Candle In The Wind 1997 / Something About The ...,ELTON JOHN,35714.0,1.0,1.0,42.0,19980328,elton john,
7,21205,161693,94,87,16.0,24.0,Holiday,MADONNA,30597.0,108.0,16.0,24.0,19840317,madonna,Madonna
8,6326,73743,44,29,1.0,24.0,Music,MADONNA,36750.0,41.0,1.0,24.0,20010120,madonna,Music
9,9546,99348,49,43,3.0,24.0,Runaway,JANET JACKSON,34958.0,6.0,3.0,24.0,19960224,janet,Design of a Decade: 1986-1996


In [35]:
billboard_top_songs_df.to_csv('billboard_top_songs_albums.csv')

In [37]:
billboard_top_songs_df['album'].value_counts()

                                                             850
Greatest Hits                                                160
The Ultimate Collection                                       45
Anthology                                                     30
The Man and His Music                                         26
ELV1S 30 #1 Hits                                              25
The Very Best Of                                              21
The Definitive Collection                                     16
Singular Genius: The Complete ABC Singles                     15
The Dionne Warwick Collection: Her All-Time Greatest Hits     14
Love Songs                                                    14
The Essential O'Jays                                          13
The Cream of Clapton                                          13
1                                                             13
Gold                                                          12
Elvis 75                 

In [7]:
response_list = []
for index,row in top_500_df.iterrows():
    artist_name = row['Artist']
    album_name = row['Album']
    last_fm_url = f'http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={api_key}&artist={artist_name}&album={album_name}&format=json'
    response = requests.get(last_fm_url)
    response_list.append(response)
    print(f'Processing album # {index}: {album_name}, by: {artist_name}: ')
    time.sleep(0.200) 

Processing album # 0: Sgt. Pepper's Lonely Hearts Club Band, by: The Beatles: 
Processing album # 1: Pet Sounds, by: The Beach Boys: 
Processing album # 2: Revolver, by: The Beatles: 
Processing album # 3: Highway 61 Revisited, by: Bob Dylan: 
Processing album # 4: Rubber Soul, by: The Beatles: 
Processing album # 5: What's Going On, by: Marvin Gaye: 
Processing album # 6: Exile on Main St., by: The Rolling Stones: 
Processing album # 7: London Calling, by: The Clash: 
Processing album # 8: Blonde on Blonde, by: Bob Dylan: 
Processing album # 9: The Beatles ("The White Album"), by: The Beatles: 
Processing album # 10: The Sun Sessions, by: Elvis Presley: 
Processing album # 11: Kind of Blue, by: Miles Davis: 
Processing album # 12: The Velvet Underground & Nico, by: The Velvet Underground: 


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))